In [10]:
# %load_ext autoreload
# %autoreload 2
PATH = '/mnt/workspace/adaptiveness_vs_learning/runtime_eval/'
import sys
sys.path.append(PATH)
from core.eval_Runtime import *

In [11]:
query_dirs = {
    'pg_stats_all': '/mnt/adaptiveness_vs_learning/queries/stats/original/all/',
}

stats_all_queries = [f'{i+1}.sql' for i in range(146)]

job_slow_queries = [
    '16b.sql', '17a.sql', '17e.sql', '17f.sql', '17b.sql', '19d.sql', '17d.sql',
    '17c.sql', '10c.sql', '26c.sql', '25c.sql', '6d.sql', '6f.sql', '8c.sql',
    '18c.sql', '9d.sql', '30a.sql', '19c.sql', '20a.sql'
]

job_rand_queries = [
    '8a.sql', '16a.sql', '2a.sql', '30c.sql', '17e.sql', '20a.sql', '26b.sql',
    '12b.sql', '15b.sql', '15d.sql', '10b.sql', '15a.sql', '4c.sql', '4b.sql',
    '22b.sql', '17c.sql', '24b.sql', '10a.sql', '22c.sql'
]

all_JOB_queries = [
    '30a.sql', '1d.sql', '25a.sql', '6b.sql', '19c.sql', '28b.sql', 
    '9b.sql', '33b.sql', '32a.sql', '27a.sql', '20b.sql', '17d.sql', 
    '16b.sql', '10b.sql', '6a.sql', '17b.sql', '25b.sql', '8b.sql', 
    '31b.sql', '33c.sql', '23a.sql', '15a.sql', '21b.sql', '11d.sql', 
    '9a.sql', '6d.sql', '24a.sql', '17e.sql', '19a.sql', '1c.sql', 
    '25c.sql', '31a.sql', '23c.sql', '17f.sql', '19b.sql', '9d.sql', 
    '27b.sql', '11b.sql', '7b.sql', '19d.sql', '1a.sql', '11c.sql', 
    '31c.sql', '28a.sql', '3c.sql', '6c.sql', '26b.sql', '16a.sql', 
    '7a.sql', '29b.sql', '29a.sql', '1b.sql', '6e.sql', '13d.sql', 
    '10c.sql', '26a.sql', '4b.sql', '10a.sql', '15b.sql', '15d.sql', 
    '26c.sql', '14b.sql', '15c.sql', '14c.sql', '13b.sql', '3a.sql', 
    '24b.sql', '5a.sql', '8c.sql', '6f.sql', '8a.sql', '27c.sql', 
    '12a.sql', '22a.sql', '17a.sql', '13a.sql', '21a.sql', '12b.sql', 
    '13c.sql', '8d.sql', '21c.sql', '7c.sql', '2a.sql', '3b.sql', 
    '16c.sql', '9c.sql', '32b.sql', '28c.sql', '33a.sql', '11a.sql', 
    '18a.sql', '5c.sql', '22d.sql', '18c.sql', '5b.sql', '2c.sql', 
    '16d.sql', '4a.sql', '22c.sql', '12c.sql', '29c.sql', '30b.sql', 
    '2d.sql', '14a.sql', '17c.sql', '22b.sql', '30c.sql', '20a.sql', 
    '20c.sql', '2b.sql', '4c.sql', '18b.sql', '23b.sql'
]

In [12]:
import sqlparse

def sql_formater(sql):
    return sqlparse.format(sql, reindent=True, keyword_case='upper')

In [13]:
engine_name = 'postgres'
db_name = 'stats'
query_dir = query_dirs['pg_stats_all']
run_queries = stats_all_queries

pg_db = DB_instance(engine_name, db_name)
pg_ev = Runtime_Evaluator(pg_db, query_dir)
pg_ev.evaluate_queries(
    sql_dir=query_dir,
    sql_files=run_queries,
    save_json_file=f'./{engine_name}_{db_name}_runtimes.json', 
    rerun_finished=False,
    disable_op="mergejoin",
    disable_parallel=False,
    multiple_runs=2,
    save_plan=False,
    timeout=6000
)

OperationalError: FATAL:  role "postgres" does not exist


In [14]:
import json
runtimes = json.load(open("./postgres_stats_runtimes.json", 'r'))

In [15]:
import numpy as np
runtimes = {q_name: np.average(runtimes[q_name]) for q_name in runtimes}

In [16]:
sorted_dict = dict(sorted(runtimes.items(), key=lambda item: -item[1]))
sorted_dict

{'120.sql': 1171.0060594081879,
 '122.sql': 1168.602886557579,
 '58.sql': 683.7813885211945,
 '68.sql': 197.63556325435638,
 '126.sql': 115.9788978099823,
 '135.sql': 85.27042031288147,
 '123.sql': 36.16596233844757,
 '34.sql': 23.672454237937927,
 '108.sql': 23.4297114610672,
 '70.sql': 22.929199695587158,
 '136.sql': 22.246567487716675,
 '107.sql': 21.98565983772278,
 '69.sql': 19.055569171905518,
 '134.sql': 12.232121229171753,
 '141.sql': 12.006289601325989,
 '143.sql': 11.994443655014038,
 '30.sql': 11.568493366241455,
 '71.sql': 11.406644463539124,
 '142.sql': 10.489479899406433,
 '52.sql': 9.78620457649231,
 '140.sql': 8.890093564987183,
 '105.sql': 7.292656660079956,
 '51.sql': 6.92228376865387,
 '56.sql': 4.747513651847839,
 '138.sql': 4.548211216926575,
 '59.sql': 4.339902877807617,
 '38.sql': 4.157012939453125,
 '115.sql': 3.9005972146987915,
 '49.sql': 3.794488549232483,
 '104.sql': 2.8248780965805054,
 '47.sql': 2.7569940090179443,
 '97.sql': 2.244305968284607,
 '113.sql':

In [17]:
query_num = len(sorted_dict)
query_set_num = int(0.2*query_num)
slow_queries = list(sorted_dict.keys())[:query_set_num]

# import random
# random.seed(0)
# random_queries = random.sample(list(sorted_dict.keys()), query_set_num)
random_queries = ['44.sql', '145.sql', '128.sql', '99.sql', '23.sql', '101.sql', '7.sql', '5.sql', '17.sql', '116.sql', '67.sql', '119.sql', '69.sql', '94.sql', '135.sql', '107.sql', '3.sql', '103.sql', '78.sql', '83.sql', '25.sql', '118.sql', '26.sql', '129.sql', '37.sql', '87.sql', '38.sql', '142.sql', '113.sql']

In [18]:
import os

for q in random_queries:
    os.system(f'cp ./all/{q} ./stats_rand/')
    with open(f'./stats_rand/{q}', 'r') as f:
        sql = f.read()
    sql = sql_formater(sql)
    with open(f'./stats_rand/{q}', 'w') as g:
        g.write(sql)

for q in stats_all_queries:
    with open(f'./all/{q}', 'r') as f:
        sql = f.read()
        
    sql = sql_formater(sql)
    if q in random_queries:
        with open(f'./balsa_stats_rand/t{q}', 'w') as g:
            g.write(sql)
    else:
        with open(f'./balsa_stats_rand/{q}', 'w') as g:
            g.write(sql)
        
for q in slow_queries:
    os.system(f'cp ./all/{q} ./stats_slow/')
    with open(f'./stats_slow/{q}', 'r') as f:
        sql = f.read()
    sql = sql_formater(sql)
    with open(f'./stats_slow/{q}', 'w') as g:
        g.write(sql)

for q in stats_all_queries:
    with open(f'./all/{q}', 'r') as f:
        sql = f.read()
    sql = sql_formater(sql)
    if q in slow_queries:
        with open(f'./balsa_stats_slow/t{q}', 'w') as g:
            g.write(sql)
    else:
        with open(f'./balsa_stats_slow/{q}', 'w') as g:
            g.write(sql)

In [19]:
total_time = 0
for q in slow_queries:
    total_time += sorted_dict[q]
print("Stats slow total time: ", total_time)

total_time = 0
for q in random_queries:
    total_time += sorted_dict[q]
print("Stats rand total time: ", total_time)


Stats slow total time:  3719.836777329445
Stats rand total time:  149.78905951976776
